In [1]:
import pandas as pd

## Preparing the Data for Cancer Type Classification

In [2]:
df_lung = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/lung.csv')
df_normal = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/normal.csv')

In [3]:
def create_combined_df(cancer_df, normal_df):
    cancer_count = len(cancer_df)
    normal_sample = normal_df.sample(n=cancer_count, random_state=8, replace=False)

    combined_df = pd.concat([cancer_df, normal_sample], ignore_index=True)
    return combined_df

In [4]:
df_lung_cancer = create_combined_df(df_lung, df_normal)

In [5]:
# Preprocessing data
# Input df, return X,y for training

def preprocessing(df):
    
    # Drop type Columns
    if "type" in df.columns:
        df = df.drop(columns="type")

    # Convert 'cancer_type' column to binary type: normal = 0, other = 1
    if 'cancer_type' in df.columns and not df['cancer_type'].isin([0, 1]).all():
        df['cancer_type'] = df['cancer_type'].map({'normal': 0}).fillna(1).astype(int)
    
    # Get X,y
    target = 'cancer_type'
    X = df.drop(columns=target)
    y = df[target]
    
    return X,y

# Display proccesed data
X_l,y_l = preprocessing(df_lung_cancer)

display(X_l,y_l)

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,12.014762,6.983442,6.540233,8.362803,3.780203,9.188556,5.674197,6.262662,9.220699,4.121583,...,13.391406,12.688067,14.222519,14.145409,10.784380,9.229371,10.030957,3.569567,4.249465,3.982578
1,11.317501,7.243950,6.927529,8.374879,3.845977,8.546901,5.680735,5.811109,8.687212,4.578750,...,13.307399,12.579227,14.192594,14.145906,9.772229,7.742450,8.501174,3.532915,4.134520,3.917711
2,10.868398,7.213200,7.110826,8.258420,4.074300,9.295490,5.494098,5.788352,10.384183,4.558008,...,13.747090,13.054736,14.391262,14.313572,11.928554,10.481570,11.281070,3.453530,4.226954,3.706964
3,11.968264,8.003929,7.167021,8.794291,3.679181,8.404464,5.877476,5.746267,8.973831,4.575594,...,12.679800,11.850551,13.872564,13.703017,4.679738,4.777636,4.329992,3.568955,4.091501,3.881763
4,11.048648,7.688655,8.113348,8.656912,3.542036,8.778009,5.740964,5.467730,10.060357,4.469260,...,13.114882,12.372716,14.019266,13.810847,4.575873,4.715638,4.442247,3.573208,3.906953,3.848166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,10.537254,7.459961,7.743787,8.062182,3.735166,8.963168,5.650002,6.039367,9.768300,4.189641,...,13.001646,12.227844,14.057034,13.984470,12.149398,10.538870,11.258644,3.815355,4.191686,3.913278
226,10.951526,6.032387,5.513643,8.808598,3.677768,8.913564,6.449267,4.852616,8.049835,3.671817,...,12.251976,11.624545,13.554286,13.401902,9.576175,5.123975,6.967157,3.287151,3.802152,3.592062
227,10.477503,7.827490,8.480160,8.595126,3.630788,8.676451,6.263245,6.034697,11.033220,4.494361,...,12.926594,12.060730,13.953663,13.810157,12.649718,6.987827,10.621531,3.871506,4.074219,4.006980
228,10.465742,7.757333,9.566909,8.579894,3.673610,8.562270,5.447044,6.850716,10.761728,4.289232,...,12.468944,11.452733,13.696956,13.636120,8.787804,6.011892,7.236161,3.574150,3.953565,3.978115


0      1
1      1
2      1
3      1
4      1
      ..
225    0
226    0
227    0
228    0
229    0
Name: cancer_type, Length: 230, dtype: int64

Feature Selection

In [6]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import SelectKBest, mutual_info_classif

def feature_selection(X, y, k_anova=500, k_mutual=500, combine_features=True):
    
    # Perform ANOVA feature selection
    anova_selector = SelectKBest(score_func=f_classif, k=k_anova)
    anova_selector.fit(X, y)
    X_anova = set(X.columns[anova_selector.get_support()])

    # Perform Mutual Information feature selection
    mutual_info_selector = SelectKBest(score_func=mutual_info_classif, k=k_mutual)
    mutual_info_selector.fit(X, y)
    X_mut = set(X.columns[mutual_info_selector.get_support()])

    selected_features = X_anova.union(X_mut)

    # Subset data with selected features
    X_reduce = X[list(selected_features)]
    return X_reduce

X_l = feature_selection(X_l, y_l, k_anova=100, k_mutual=100, combine_features=True)

## Training the Models and Returning their LOOCV Score

In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, f1_score

# Function Definition
def perform_model(X, y, max_iter=1000):
    model = LogisticRegression(penalty='l1', solver='saga', max_iter=max_iter, random_state=42)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    accuracy_scores, recall_scores, f1_scores = [], [], []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy_scores.append(accuracy_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, zero_division=0))

    # Train the final model on the entire dataset
    model.fit(X, y)

    # Print cross-validation results
    print("\nCross-Validation Results:")
    print(f"Mean Accuracy: {np.mean(accuracy_scores):.2f} ± {np.std(accuracy_scores):.2f}")
    print(f"Mean Recall: {np.mean(recall_scores):.2f} ± {np.std(recall_scores):.2f}")
    print(f"Mean F1 Score: {np.mean(f1_scores):.2f} ± {np.std(f1_scores):.2f}")

# Return the model and summary results
    return {
        "model": model
    }

In [8]:
result = perform_model(X_l, y_l, max_iter=5000)
result

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Cross-Validation Results:
Mean Accuracy: 0.98 ± 0.02
Mean Recall: 0.98 ± 0.03
Mean F1 Score: 0.98 ± 0.02


{'model': LogisticRegression(max_iter=5000, penalty='l1', random_state=42, solver='saga')}

In [9]:
model = result["model"]

In [10]:
df_test = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/test_data.csv')
df_lung_test = df_test[df_test['cancer_type'].isin(['normal', 'lung'])]

In [11]:
X_test,y_test = preprocessing(df_lung_test)
X_test = X_test[X_l.columns]

In [12]:
from sklearn.metrics import classification_report

# Predict using the final trained model
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print results
print("\nFinal Model Evaluation on Test Dataset:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Final Model Evaluation on Test Dataset:
Accuracy: 0.98
Recall: 1.00
F1 Score: 0.96

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        52
           1       0.92      1.00      0.96        11

    accuracy                           0.98        63
   macro avg       0.96      0.99      0.97        63
weighted avg       0.99      0.98      0.98        63



In [13]:
# Save model

import joblib
joblib.dump(model, 'lung.joblib')

['lung.joblib']